## KNN Neighbours 

In [66]:
!pip3 install nbimporter

Let´s import the functions from the cleaning notebook

In [67]:
import nbimporter
from Cleaning_Dataset import clean_diamonds, standard

In [93]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
from sklearn.model_selection import learning_curve, GridSearchCV

In [69]:
original_train = pd.read_csv("../Data/train.csv")

In [70]:
original_test = pd.read_csv("../Data/test.csv")

In [71]:
original_test.sample(5)

,id,carat,cut,color,clarity,depth,table,x,y,z
2012,2012,0.73,Ideal,E,SI1,62.2,56.0,5.74,5.78,3.58
6994,6994,0.70,Ideal,J,SI2,61.6,57.0,5.68,5.65,3.49
8542,8542,0.31,Premium,G,IF,61.6,55.0,4.42,4.38,2.71
9104,9104,1.18,Ideal,J,VS1,61.7,56.0,6.79,6.82,4.20
9902,9902,1.00,Ideal,E,SI1,62.6,56.0,6.37,6.29,3.96


In [120]:
original_train.sample(5)

,id,carat,depth,table,x,y,z,price,COLOR,CUT,CLARITY
390,390,2.02,61.7,58.0,8.15,8.11,5.02,9.825,5,4,2
3904,3904,0.30,63.0,55.0,4.31,4.29,2.71,6.515,4,5,4
7651,7651,0.31,61.5,55.0,4.34,4.37,2.68,6.967,5,5,8
18844,18844,0.35,61.6,56.0,4.58,4.54,2.81,6.564,2,5,5
23473,23473,0.70,60.5,59.0,5.71,5.76,3.47,7.922,6,3,4


Let´s call our cleaning functions giving the train data as the argument

In [72]:
original_train_cleaned = clean_diamonds(original_train)

Let´s check the cleaning process it´s done correctly

In [73]:
original_train_cleaned.sample(5)

,id,carat,depth,table,x,y,z,price,COLOR,CUT,CLARITY
32482,32482,0.50,61.8,54.1,5.10,5.14,3.16,7.878,7,5,6
25130,25130,0.70,60.1,63.0,5.68,5.71,3.42,7.958,6,2,6
4040,4040,0.90,63.7,56.0,6.10,6.06,3.87,8.031,4,2,3
29354,29354,0.39,61.9,57.0,4.69,4.68,2.90,7.268,5,5,8
26776,26776,0.70,63.4,61.0,5.62,5.58,3.55,7.685,3,3,3


Now let´s standardize the data

In [74]:
final_train = standard(original_train_cleaned)

Let´s drop the id column, which we have repeated in the index

In [76]:
final_train.drop("id", axis = 1, inplace = True)

Let´s check is working

In [77]:
final_train.sample(5)

,carat,depth,table,x,y,z,price,COLOR,CUT,CLARITY
22790,-0.519412,0.103285,-0.650557,-0.415023,-0.339420,-0.388681,-0.120574,-0.830642,0.981412,2.407127
7946,2.936219,0.590877,0.246062,2.255567,2.191154,2.296644,1.665355,-2.007344,0.085707,-1.250418
20516,0.455797,-2.473987,1.590990,0.665717,0.702581,0.346250,0.539165,0.346059,-1.705703,-1.250418
19788,-0.243810,-0.036027,-1.098866,-0.013095,-0.059218,-0.035349,0.052004,-0.242292,0.981412,0.578354
10207,-1.134218,0.381909,-1.547175,-1.460037,-1.425203,-1.392145,-1.772385,-0.242292,0.981412,-0.031237


### Model Building 

Firstly, let´s divide the final_train into train and test, to train and test our model.

In [101]:
X = final_train.drop("price", axis = 1)
y = final_train["price"]
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.20, 
                                                    random_state=123)

In [108]:
n_neighbors = np.arange(1,15)

In [109]:
model = KNeighborsRegressor()
parameter_space = {"n_neighbors": n_neighbors}
#We are going to try with k from 1 to 10, and find the best possible k with grid search
grid_search = GridSearchCV(model,param_grid = parameter_space,
                          cv = 5) #cross validation 
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])})

In [110]:
grid_search.best_score_

0.9814479301059192

In [111]:
best_knn = grid_search.best_estimator_

In [112]:
best_knn.score(X_test, y_test)

0.9808214272211832

Now that we now this is the best possible KNN model, let´s train it with all the data we have available

In [113]:
best_knn.fit(X, y)

KNeighborsRegressor(n_neighbors=11)

### Real test

Now we are going to estimate on the real test data set, in which we lack the information of the target variable

Firstly, we need to clean and standardize the data set:

In [117]:
original_test_cleaned = clean_diamonds(original_test)

In [118]:
final_test = standard(original_test_cleaned)

In [121]:
final_test.drop("id", axis = 1, inplace = True)

In [123]:
prediction = best_knn.predict(final_test)

In [129]:
len(prediction)

13485

The results don´t make any sense because they have been standardrized, let´s try to reverse this process. 

Standardization is done by subtracting the average and dividing the difference by the standard deviation.

yi=xi−x¯s

To reverse the effect: xi=yi×s+x¯

https://gist.github.com/thijsessens/f2ac3dc68ef8a349a723444cb012c821

In [136]:
#Tenemos que hacer el proceso que ha ocurrido en la funcion de Standard fuera de la función para poder revertirla
scaler = StandardScaler()
scaler.fit(original_train_cleaned["price"].values.reshape(-1, 1))
real_prediction = scaler.inverse_transform(prediction)

In [137]:
real_prediction

array([6.79527273, 6.83236364, 8.28245455, ..., 6.58472727, 7.27454545,
       8.14227273])

In [138]:
len(real_prediction)

13485

Let´s see a sample submission

In [124]:
sample_submission = pd.read_csv("../Data/sample_submission.csv")

In [139]:
sample_submission.head(5)

,id,price
0,0,6.033588
1,1,6.880996
2,2,8.871935
3,3,7.250187
4,4,5.855043


In [130]:
len(sample_submission)

13485

In [140]:
KNN_df = pd.DataFrame(real_prediction, columns=["price"])

In [158]:
KNN_df["id"] = KNN_df.index

In [159]:
KNN_df = KNN_df[["id", "price"]]

In [162]:
sample_submission.shape

(13485, 2)

In [161]:
KNN_df.shape

(13485, 2)

In [165]:
sample_submission.head(1)

,id,price
0,0,6.033588


In [166]:
KNN_df.head(1)

,id,price
0,0,6.795273


In [167]:
KNN_df.to_csv(r"KNN_df.csv", index = False)

In [168]:
!mv KNN_df.csv ../Predictions/KNN_df.csv